In [2]:
import pandas as pd

Utilizando o metodo `%run` para rodar as funções gerais, que poderão ser utilizadas pelo código

In [3]:
%run funcoes_gerais.ipynb

Importando as tabelas referentes aos ano

In [4]:
PAM_2017 = pd.read_excel("../dataset/Producao Alimentar/Tipos de Alimento/PAM_2017.xlsx")

In [5]:
pib_Munic = pd.read_excel("../dataset/Economico/Renda e Desenvolvimento/PIB/PIB-Munic.xlsx")
pib_Munic_BD = pd.read_excel("../dataset/Economico/Renda e Desenvolvimento/PIB/PIB dos Municбpios - base de dados 2010-2021.xlsx")

In [6]:
cod_municipios = pd.read_csv('../Código dos Municípios')

In [7]:
censo_agro = pd.read_excel("../dataset/Producao Alimentar/Tipos de Cultivo e Produtores/Tabela 6906.ods")

# Analise Inicial

### Censo agro

In [8]:
censo_agro

,Município,Total,Agricultura familiar - não,Agricultura familiar - sim
0,Alta Floresta D'Oeste (RO),2582,566,2016
1,Ariquemes (RO),2303,543,1760
2,Cabixi (RO),961,187,774
3,Cacoal (RO),3438,411,3027
4,Cerejeiras (RO),671,203,468
...,...,...,...,...
5558,Vianópolis (GO),799,185,614
5559,Vicentinópolis (GO),120,69,51
5560,Vila Boa (GO),74,54,20
5561,Vila Propício (GO),629,252,377


### PIB Munic BD

O banco de dados do PIB Munic possui dados de todos os anos, então separamos os dados referentes ao ano de 2017

Os dados retirados da tabela são referentes ao `Código do Município`, `Produto Interno Bruto per capita, \na preços correntes\n(R$ 1,00)` que se refere ao PIB do município nos preços correntes(preços correntes são os valores de bens e serviços são referentes ao ano em causa) e `Atividade com maior valor adicionado bruto` que informa a atividade com maior investimento do município

In [9]:
pib_Munic_BD_2017 = pib_Munic_BD[pib_Munic_BD['Ano'] == 2017][['Código do Município', 'Produto Interno Bruto per capita, \na preços correntes\n(R$ 1,00)','Atividade com maior valor adicionado bruto']]

In [10]:
pib_Munic_BD_2017.head()

,Código do Município,"Produto Interno Bruto per capita, \na preços correntes\n(R$ 1,00)",Atividade com maior valor adicionado bruto
38975,1100015,19081.43,"Administração, defesa, educação e saúde públic..."
38976,1100023,21313.62,Demais serviços
38977,1100031,22189.90,"Administração, defesa, educação e saúde públic..."
38978,1100049,23532.16,Demais serviços
38979,1100056,24492.33,"Administração, defesa, educação e saúde públic..."


In [11]:
pib_Munic_BD_2017.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5570 entries, 38975 to 44544
Data columns (total 3 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   Código do Município                                              5570 non-null   int64  
 1   Produto Interno Bruto per capita, 
a preços correntes
(R$ 1,00)  5570 non-null   float64
 2   Atividade com maior valor adicionado bruto                       5570 non-null   object 
dtypes: float64(1), int64(1), object(1)
memory usage: 174.1+ KB


# Transformação

#### Censo Agro

Utiliza-se  afunção de separar nome da UF

In [12]:
Separa_Nome_UF(censo_agro)

depois adicionamos o código do município e o numero da UF

In [13]:
censo_agro = censo_agro.merge(cod_municipios[['Codigo_Municipio', 'UF', 'Município', 'Sigla_UF']], how='right',left_on=['Município', 'Sigla_UF'], right_on=['Município', 'Sigla_UF'])

In [14]:
censo_agro = censo_agro[['Codigo_Municipio', 'Município', 'Sigla_UF', 'UF', 'Agricultura familiar - sim', 'Agricultura familiar - não', 'Total']]
censo_agro

,Codigo_Municipio,Município,Sigla_UF,UF,Agricultura familiar - sim,Agricultura familiar - não,Total
0,1100015,Alta Floresta D'Oeste,RO,11,2016,566,2582
1,1100379,Alto Alegre dos Parecis,RO,11,1308,349,1657
2,1100403,Alto Paraíso,RO,11,1964,318,2282
3,1100346,Alvorada D'Oeste,RO,11,1396,274,1670
4,1100023,Ariquemes,RO,11,1760,543,2303
...,...,...,...,...,...,...,...
5565,5222005,Vianópolis,GO,52,614,185,799
5566,5222054,Vicentinópolis,GO,52,51,69,120
5567,5222203,Vila Boa,GO,52,20,54,74
5568,5222302,Vila Propício,GO,52,377,252,629


#### PIB MUNIC

Substituição do modelo de unidade monetaria Americana para o Brasileiro

In [15]:
for col in pib_Munic.columns[2:]:
    pib_Munic[col] = pib_Munic[col].apply(lambda x: f"{x:,.1f}".replace(",", "X").replace(".", ",").replace("X", "."))

Substituição dos nome das colunas para melhor utilização e visualização

In [16]:
pib_Munic = pib_Munic.rename(columns={'Cód.':'Codigo_Municipio','2017':'PIB do Municipio'})

In [17]:
pib_Munic.head()

,Codigo_Municipio,Município,PIB do Municipio,2018,2019,2020,2021
0,1100015,Alta Floresta D'Oeste (RO),"485.374,0","498.980,0","495.775,0","570.242,0","734.469,0"
1,1100023,Ariquemes (RO),"2.287.910,0","2.464.704,0","2.579.278,0","2.817.331,0","3.209.761,0"
2,1100031,Cabixi (RO),"138.110,0","140.503,0","139.976,0","167.153,0","238.412,0"
3,1100049,Cacoal (RO),"2.082.761,0","2.175.840,0","2.261.930,0","2.518.845,0","2.792.383,0"
4,1100056,Cerejeiras (RO),"439.245,0","470.647,0","506.494,0","600.630,0","743.037,0"


#### PIB Munic BD

In [18]:
pib_Munic_BD_2017.head()

,Código do Município,"Produto Interno Bruto per capita, \na preços correntes\n(R$ 1,00)",Atividade com maior valor adicionado bruto
38975,1100015,19081.43,"Administração, defesa, educação e saúde públic..."
38976,1100023,21313.62,Demais serviços
38977,1100031,22189.90,"Administração, defesa, educação e saúde públic..."
38978,1100049,23532.16,Demais serviços
38979,1100056,24492.33,"Administração, defesa, educação e saúde públic..."


Alteração dos nomes das colunas e formatação do sistema monetário

In [19]:
pib_Munic_BD_2017 = pib_Munic_BD_2017.rename(columns={'Código do Município': 'Codigo_Municipio', 
                        'Produto Interno Bruto per capita, \na preços correntes\n(R$ 1,00)': 'Produto Interno Bruto per capita, a preços correntes(R$ 1,00)'})
pib_Munic_BD_2017['Produto Interno Bruto per capita, a preços correntes(R$ 1,00)'] = pib_Munic_BD_2017['Produto Interno Bruto per capita, a preços correntes(R$ 1,00)'].apply(lambda x: f"{x:,.1f}".replace(",", "X").replace(".", ",").replace("X", "."))

#### PAM

Utilização da função de alteração da PAM

In [20]:
PAM_2017 = PAM(PAM_2017)

# Integração

Inicio da integração de fato, aqui utilizaremos a função `merge` que se basea no JOIN da linguagem SQL, nela temos que passar uma variavel chave que vai ligar os outros elementos da tabela

In [21]:
bd_2017 = cod_municipios[['Codigo_Municipio', 'Município', 'UF', 'Sigla_UF']]

In [22]:
bd_2017.head()

,Codigo_Municipio,Município,UF,Sigla_UF
0,1100015,Alta Floresta D'Oeste,11,RO
1,1100379,Alto Alegre dos Parecis,11,RO
2,1100403,Alto Paraíso,11,RO
3,1100346,Alvorada D'Oeste,11,RO
4,1100023,Ariquemes,11,RO


In [23]:
bd_2017 = bd_2017.merge(censo_agro, how='right')

In [24]:
bd_2017 = bd_2017.merge(pib_Munic[['Codigo_Municipio', 'PIB do Municipio']], how='right')

In [25]:
bd_2017 = bd_2017.merge(pib_Munic_BD_2017, how='right')

In [26]:
bd_2017 = bd_2017.merge(PAM_2017, how='right')

In [27]:
bd_2017 = bd_2017.rename(columns={'Agricultura familiar - sim': 'Número de estabelecimentos agropecuários considerados como agricultura familiar', 
                         'Agricultura familiar - não': 'Número de estabelecimentos agropecuários não considerados como agricultura familiar',
                         'Total': 'Número total de estabelecimentos agropecuários'})

In [28]:
bd_2017 

,Codigo_Municipio,Município,UF,Sigla_UF,Número de estabelecimentos agropecuários considerados como agricultura familiar,Número de estabelecimentos agropecuários não considerados como agricultura familiar,Número total de estabelecimentos agropecuários,PIB do Municipio,"Produto Interno Bruto per capita, a preços correntes(R$ 1,00)",Atividade com maior valor adicionado bruto,...,Produção de Sisal ou agave (fibra)(toneladas),Produção de Soja (em grão)(toneladas),Produção de Sorgo (em grão)(toneladas),Produção de Tangerina(toneladas),Produção de Tomate(toneladas),Produção de Trigo (em grão)(toneladas),Produção de Triticale (em grão)(toneladas),Produção de Tungue (fruto seco)(toneladas),Produção de Urucum (semente)(toneladas),Produção de Uva(toneladas)
0,1100015,Alta Floresta D'Oeste,11,RO,2016,566,2582,"485.374,0","19.081,4","Administração, defesa, educação e saúde públic...",...,-,1350,-,-,-,-,-,-,-,-
1,1100379,Alto Alegre dos Parecis,11,RO,1308,349,1657,"328.001,0","23.353,6","Agricultura, inclusive apoio à agricultura e a...",...,-,10135,-,63,680,-,-,-,2,-
2,1100403,Alto Paraíso,11,RO,1964,318,2282,"307.459,0","14.699,7","Administração, defesa, educação e saúde públic...",...,-,21882,-,-,4,-,-,-,-,-
3,1100346,Alvorada D'Oeste,11,RO,1396,274,1670,"252.252,0","15.062,5","Administração, defesa, educação e saúde públic...",...,-,3036,-,-,-,-,-,-,-,-
4,1100023,Ariquemes,11,RO,1760,543,2303,"2.287.910,0","21.313,6",Demais serviços,...,-,10959,-,-,-,-,-,-,4,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5565,5222005,Vianópolis,52,GO,614,185,799,"395.861,0","28.947,8",Demais serviços,...,-,129500,2000,-,70000,855,-,-,-,-
5566,5222054,Vicentinópolis,52,GO,51,69,120,"350.839,0","41.781,4","Agricultura, inclusive apoio à agricultura e a...",...,-,79200,16000,-,23250,-,-,-,-,-
5567,5222203,Vila Boa,52,GO,20,54,74,"118.639,0","20.701,3","Administração, defesa, educação e saúde públic...",...,-,8250,900,-,-,-,-,-,-,-
5568,5222302,Vila Propício,52,GO,377,252,629,"182.203,0","32.021,6","Agricultura, inclusive apoio à agricultura e a...",...,-,45050,-,-,3720,-,-,-,-,-


In [29]:
bd_2017.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 81 columns):
 #   Column                                                                               Non-Null Count  Dtype 
---  ------                                                                               --------------  ----- 
 0   Codigo_Municipio                                                                     5570 non-null   int64 
 1   Município                                                                            5570 non-null   object
 2   UF                                                                                   5570 non-null   int64 
 3   Sigla_UF                                                                             5570 non-null   object
 4   Número de estabelecimentos agropecuários considerados como agricultura familiar      5563 non-null   object
 5   Número de estabelecimentos agropecuários não considerados como agricultura familiar  5563 non-nul

In [30]:
bd_2017.describe()

,Codigo_Municipio,UF
count,5.570000e+03,5570.000000
mean,3.253591e+06,32.377738
std,9.849103e+05,9.833862
min,1.100015e+06,11.000000
25%,2.512126e+06,25.000000
50%,3.146280e+06,31.000000
75%,4.119190e+06,41.000000
max,5.300108e+06,53.000000
